In [1]:
from IPython.core.interactiveshell import InteractiveShell
from pandas import Timestamp
import numpy as np
from numba import njit
import numba

# import pycurl_requests as requests
import requests
import pandas as pd
from pprint import pprint
import sys
import itertools
import snoop
import json

sys.path('a')

sys.path.append(
    "/Users/GlebSokolov/.vscode/extensions/almenon.arepl-2.0.3/node_modules/arepl-backend/python"
)
# from arepl_dump import dump

InteractiveShell.ast_node_interactivity = "all"


In [2]:
import coinbase
from coinbase.wallet.model import APIObject
from coinbase.wallet.client import Client
import cbpro

coinbase_API_key = "PZsUB9TdWedNjJcu"
coinbase_API_secret = "UP8Jz2cfXvDRdgeeNwFy6fCxfJCkQFea"
client = Client(coinbase_API_key, coinbase_API_secret)


In [3]:
import time
import numpy as np

symbol = "AGLD-USDT"

since = "2021-10-05 10:00:00"
end = "2021-10-05 11:00:00"
since_ts = int(Timestamp(since).timestamp() * 1000)
end_ts = int(Timestamp(end).timestamp() * 1000)


In [24]:
%%capture
import cbpro
public_client = cbpro.PublicClient()
r = public_client.get_product_order_book('BTC-USD', level=1)
# responses_ = list(public_client.get_product_trades(product_id=symbol))


In [25]:
r

{'bids': [['51363.72', '0.05', 1]],
 'asks': [['51363.73', '0.04795559', 4]],
 'sequence': 32507063554,
 'auction_mode': False,
 'auction': None}

In [4]:
# @njit
# %%snoop
def find_trades(responses_, ts_start, ts_end):
    trade_ids = np.zeros(len(responses_), dtype=int)
    rrr = np.zeros(len(responses_), dtype=np.int64)
    for i in range(rrr.shape[0]):
        rrr[i] = int(Timestamp(responses_[i]["time"]).timestamp()) * 1000
        trade_ids[i] = responses_[i]["trade_id"]
    ids = []
    for i in range(len(rrr)):
        if ts_start <= rrr[i] <= ts_end:
            ids.append((trade_ids[i], i))
    id_start, id_end = ids[-1][0], ids[0][0]
    return [i for i in responses_ if id_start <= i["trade_id"] <= id_end]



In [5]:
class CurrencyData:
    def __init__(self, currency, ts, te):
        self.binance = []
        self.coinbase = []
        self.currency = currency
        self.ts = ts
        self.te = te

    # @snoop
    def get_coinbase(self):
        public_client.get_product_ticker(product_id=self.currency + "-USD")
        responses_ = list(
            public_client.get_product_trades(product_id=self.currency + "-USDT")
        )
        self.coinbase = find_trades(responses_, self.ts, self.te)

    def get_binance(self):
        self.binance = requests.get(
            "https://api.binance.com/api/v3/aggTrades",
            params={
                "symbol": self.currency + "USDT",
                "startTime": self.ts,
                "endTime": self.te,
            },
        ).json()
        # return btrades

    def coinbase_sum(self):
        self.coinbase = pd.DataFrame(self.coinbase)
        self.coinbase["time"] = self.coinbase["time"].apply(lambda x: pd.to_datetime(x))
        intid = pd.interval_range(
            start=pd.to_datetime(since), end=pd.to_datetime(end), freq="T"
        )
        self.coinbase["bins"] = pd.cut(self.coinbase["time"], intid)
        self.coinbase = self.coinbase.set_index("bins")
        self.coinbase["sum"] = self.coinbase["size"].astype("float") * self.coinbase[
            "price"
        ].astype("float")
        self.coinbase["size"] = self.coinbase["size"].astype("float")
        self.cb_grouped = self.coinbase.groupby(["bins", "side"])["sum"].sum()
        return self.cb_grouped

    def binance_sum(self):
        self.binance = pd.DataFrame(self.binance)
        self.binance["T"] = self.binance["T"].apply(
            lambda x: pd.to_datetime(x, unit="ms")
        )
        intid = pd.interval_range(
            start=pd.to_datetime(since), end=pd.to_datetime(end), freq="T"
        )
        self.binance["bins"] = pd.cut(self.binance["T"], intid)
        self.binance = self.binance.set_index("bins")
        self.binance["sum"] = self.binance["q"].astype("float") * self.binance[
            "p"
        ].astype("float")
        self.binance["q"] = self.binance["q"].astype("float")
        self.bn_grouped = self.binance.groupby(["bins", "m"])["sum"].sum()
        return self.bn_grouped

    def calculate_spread(self):
          


In [14]:
agld.coinbase

,time,trade_id,price,size,side,sum
bins,,,,,,
"(2021-10-05 10:55:00, 2021-10-05 10:56:00]",2021-10-05 10:55:29.249503+00:00,15582,3.64000000,62.30,buy,226.7720
"(2021-10-05 10:53:00, 2021-10-05 10:54:00]",2021-10-05 10:53:57.829334+00:00,15581,3.59000000,12.23,buy,43.9057
"(2021-10-05 10:53:00, 2021-10-05 10:54:00]",2021-10-05 10:53:57.829334+00:00,15580,3.59000000,208.49,buy,748.4791
"(2021-10-05 10:53:00, 2021-10-05 10:54:00]",2021-10-05 10:53:57.829334+00:00,15579,3.59000000,6.45,buy,23.1555
"(2021-10-05 10:53:00, 2021-10-05 10:54:00]",2021-10-05 10:53:57.829334+00:00,15578,3.59000000,4.41,buy,15.8319
"(2021-10-05 10:51:00, 2021-10-05 10:52:00]",2021-10-05 10:51:09.028731+00:00,15577,3.63000000,6.08,buy,22.0704
"(2021-10-05 10:50:00, 2021-10-05 10:51:00]",2021-10-05 10:50:49.420777+00:00,15576,3.64000000,32.69,buy,118.9916
"(2021-10-05 10:50:00, 2021-10-05 10:51:00]",2021-10-05 10:50:03.368874+00:00,15575,3.66000000,5.39,sell,19.7274
"(2021-10-05 10:47:00, 2021-10-05 10:48:00]",2021-10-05 10:47:09.734114+00:00,15574,3.65000000,228.76,sell,834.9740


In [31]:
agld.binance

,a,p,q,f,l,T,m,M,sum
bins,,,,,,,,,
"(2021-10-05 10:00:00, 2021-10-05 10:01:00]",47980,3.73900000,2.6,67762,67762,2021-10-05 10:00:00.218,False,True,9.7214
"(2021-10-05 10:00:00, 2021-10-05 10:01:00]",47981,3.73900000,513.6,67763,67763,2021-10-05 10:00:01.019,False,True,1920.3504
"(2021-10-05 10:00:00, 2021-10-05 10:01:00]",47982,3.74000000,55.3,67764,67765,2021-10-05 10:00:01.714,False,True,206.8220
"(2021-10-05 10:00:00, 2021-10-05 10:01:00]",47983,3.74100000,94.8,67766,67766,2021-10-05 10:00:01.714,False,True,354.6468
"(2021-10-05 10:00:00, 2021-10-05 10:01:00]",47984,3.74200000,321.6,67767,67782,2021-10-05 10:00:01.714,False,True,1203.4272
...,...,...,...,...,...,...,...,...,...
"(2021-10-05 10:59:00, 2021-10-05 11:00:00]",52234,3.65200000,346.0,73903,73903,2021-10-05 10:59:45.877,False,True,1263.5920
"(2021-10-05 10:59:00, 2021-10-05 11:00:00]",52235,3.65000000,41.6,73904,73905,2021-10-05 10:59:46.953,True,True,151.8400
"(2021-10-05 10:59:00, 2021-10-05 11:00:00]",52236,3.65000000,5.6,73906,73906,2021-10-05 10:59:48.912,True,True,20.4400


In [6]:
agld = CurrencyData("AGLD", since_ts, end_ts)
agld.get_binance()
agld.binance_sum()

bins                                        m    
(2021-10-05 10:00:00, 2021-10-05 10:01:00]  False    95060.9329
                                            True     41135.4677
(2021-10-05 10:01:00, 2021-10-05 10:02:00]  False    20183.8912
                                            True      8594.1856
(2021-10-05 10:02:00, 2021-10-05 10:03:00]  False    20115.8112
                                                        ...    
(2021-10-05 10:57:00, 2021-10-05 10:58:00]  True     15370.5811
(2021-10-05 10:58:00, 2021-10-05 10:59:00]  False     7180.2872
                                            True     10124.5458
(2021-10-05 10:59:00, 2021-10-05 11:00:00]  False     9910.0520
                                            True      2030.6505
Name: sum, Length: 120, dtype: float64

In [21]:
public_client.get_product_order_book('BTC-USD', level=1)

NameError: name 'public_client' is not defined

In [20]:
import requests

host = "wss://api.gateio.ws"
prefix = "/ws/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

url = '/spot/trades'
query_param = 'currency_pair=BTC_USDT'
r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
# print(r.json())
gio = pd.DataFrame(r.json())

InvalidSchema: No connection adapters were found for 'wss://api.gateio.ws/ws/v4/spot/trades?currency_pair=BTC_USDT'

In [19]:
from __future__ import print_function

import gate_api
from gate_api.exceptions import ApiException, GateApiException

configuration = gate_api.Configuration(
    host = "https://api.gateio.ws/api/v4"
)


api_client = gate_api.ApiClient(configuration)
api_instance = gate_api.SpotApi(api_client)
settle = 'usdt' # str | Settle currency

api_response = api_instance.list_trades('AGLD_USDT', reverse=False, _from=since_ts, to=end_ts, page=1)

GateApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 27 Dec 2021 13:11:11 GMT', 'Content-Type': 'application/json', 'Content-Length': '62', 'Connection': 'keep-alive', 'Server': 'openresty', 'X-Request-Id': '[9d42d922-2409364892]'})
HTTP response body: {"label":"INVALID_PARAM_VALUE","message":"invalid time range"}


In [18]:
int(gio.create_time[0])/since_ts

0.0010043969308717617

In [9]:
gio

,id,create_time,create_time_ms,currency_pair,side,amount,price
0,2500314951,1640610070,1640610069930.620800,BTC_USDT,buy,0.0016,51143.9
1,2500313803,1640610053,1640610052894.658200,BTC_USDT,buy,0.0012,51143.9
2,2500313616,1640610050,1640610050470.998000,BTC_USDT,sell,0.0029,51143.89
3,2500313559,1640610049,1640610049144.637200,BTC_USDT,sell,0.0029,51143.89
4,2500313281,1640610045,1640610044802.076200,BTC_USDT,sell,0.0029,51143.89
...,...,...,...,...,...,...,...
95,2500308092,1640609982,1640609981917.790000,BTC_USDT,sell,0.0165,51129.1
96,2500307568,1640609975,1640609975363.388200,BTC_USDT,sell,0.0001,51129.1
97,2500307562,1640609975,1640609975340.820000,BTC_USDT,sell,0.0122,51129.1
98,2500307560,1640609975,1640609975332.765100,BTC_USDT,sell,0.0019,51131.37
